<a href="https://colab.research.google.com/github/jenbrox/INFX497/blob/main/Housing3.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import os
print("my path is ", os.getcwd())

my path is  c:\Users\JB\infx497\ml\ml_env


# Chapter 2
Your first task is to use California census data to build a model of housing prices in the state. This data includes metrics such as the population, median income, and median housing price for each
dstrict in California. Your model should learn from this data and be able to predict the median housing price in any district, given all the other metrics.

## Machine Learning Project Checklist

1. Look at the big picture.
2. Get the data.
3. Discover and visualize the data to gain insights.
4. Prepare the data for Machine Learning algorithms.
5. Select a model and train it.
6. Fine-tune your model.
7. Present your solution.
8. Launch, monitor, and maintain your system.

### 1. Frame the problem and look at the big picture
Model's output, a prediction of district's median housing price, will be ged to another ML system (pipeline) along with other information which will determine whether it is worth investing in the area or not.

Ask yourself, is this a Supervised, Unsupervised, or Reinforcement Learning problem? Is it a calssification task, regression task, or something else? Should you use batch learning, online learning, or something else?

1. Supervised
- Regression: predicting a value
- Classification: Predict a class
2. Unsupervised
- Clustering: Group ungrouped data
- Assocation Rule Mining: Find associations between two or more classes
3. Reinforcement
- Supervised univariate regression task

In [ ]:
#function that fetches housing.tgz data
import os
import tarfile
import urllib


DOWNLOAD_ROOT = "https://raw.githubusercontent.com/ageron/handson-ml2/master/"
HOUSING_PATH = os.path.join("datasets", "housing")
HOUSING_URL = DOWNLOAD_ROOT + "datasets/housing/housing.tgz"

#creates a datasets/housing directory in your workspace, downloads the housing.tgz file, and extracts the housing.csv file from it in this directory
def fetch_housing_data(housing_url=HOUSING_URL, housing_path="HOUSING_PATH"):
    os.makedirs(housing_path, exist_ok=True)
    tgz_path = os.path.join(housing_path, "housing.tgz")
    urllib.request.urlretrieve(housing_url, tgz_path)
    housing_tgz = tarfile.open(tgz_path)
    housing_tgz.extractall(path=housing_path)
    housing_tgz.close()

In [ ]:
#load the data using pandas
import pandas as pd

#returns a pandas DataFrame object containing all the data
def load_housing_data(housing_path=HOUSING_PATH):
    csv_path = os.path.join(housing_path, "housing.csv")
    return pd.read_csv(csv_path)

C:\Users\JB\AppData\Local\Temp\ipykernel_8860\1200238656.py:2: DeprecationWarning: 
Pyarrow will become a required dependency of pandas in the next major release of pandas (pandas 3.0),
(to allow more performant data types, such as the Arrow string type, and better interoperability with other libraries)
but was not found to be installed on your system.
If this would cause problems for you,
please provide us feedback at https://github.com/pandas-dev/pandas/issues/54466
        
  import pandas as pd


In [ ]:
#view top five rows in the dataset using head()
#10 attributes
housing = load_housing_data()
housing.head()

FileNotFoundError: [Errno 2] No such file or directory: 'datasets\\housing\\housing.csv'

In [ ]:
#get a quick description of the data using info()
housing.info()

In [ ]:
#only ocean_proximity is an object, but we know it is text since this is loaded from a CSV file
#find out what categories exist and how many districts belong to each category using value_counts()
print(housing["ocean_proximity"].value_counts())

In [ ]:
#summary of numerical attributes using describe() to get a feel for the data
housing.describe()

In [ ]:
#plot a histogram to get a feel for the data
#plot a histogram for each numerical attribute. A histogram shows the number of instances (on the vertical axis) that have a given value range (on the horizontal axis).
#specify which backend Matplotlib, tells Jupyter to set up Matplotlib so it uses Jupyter’s own backend.
# only in a Jupyter notebook
%matplotlib inline
import matplotlib.pyplot as plt
housing.hist(bins=50, figsize=(20,15))
#optional in a Jupyter notebook, as Jupyter will automatically display plots when a cell is executed
plt.show()

# Create a test set
Pick instances randomly, typically %20 of the testset. Don't look at data too much before setting aside some training data even before choosing an algorithm to avoid your own biases

In [ ]:
import numpy as np

def split_train_test(data, test_ratio):
 shuffled_indices = np.random.permutation(len(data))
 test_set_size = int(len(data) * test_ratio)
 test_indices = shuffled_indices[:test_set_size]
 train_indices = shuffled_indices[test_set_size:]
 return data.iloc[train_indices], data.iloc[test_indices]

In [ ]:
#compute a hash of each instance’s identifier and put that instance in the test set if the hash is lower than or equal to
#20% of the maximum hash value. This ensures that the test set will remain consistent across multiple runs, even if you refresh the dataset.
#The new test set will contain 20% of the new instances, but it will not contain any instance that was previously in the training set.
from zlib import crc32

def test_set_check(identifier, test_ratio):
 return crc32(np.int64(identifier)) & 0xffffffff < test_ratio * 2**32

def split_train_test_by_id(data, test_ratio, id_column):
 ids = data[id_column]
 in_test_set = ids.apply(lambda id_: test_set_check(id_, test_ratio))
 return data.loc[~in_test_set], data.loc[in_test_set]

In [ ]:
#adds an `index` column
# housing_with_id["id"] = housing["longitude"] * 1000 + housing["latitude"]
# train_set, test_set = split_train_test_by_id(housing_with_id, 0.2, "id")

In [ ]:
#adds an 'index' column with sklearn with more features than above
#random_state parameter that allows you to set the random generator seed
#can pass it multiple datasets with an identical number of rows, and it will split them on the same indices
from sklearn.model_selection import train_test_split
train_set, test_set = train_test_split(housing, test_size=0.2, random_state=42)

In [ ]:
#To reduce sampling bias, we implement strata samplying and ensure our data includes the median income to accurately predict median home prices
#using the pd.cut() function to create an income category attribute with five categories (labeled from 1 to 5): category 1 ranges from 0 to 1.5
#(i.e., less than $15,000), category 2 from 1.5 to 3, etc
housing["income_cat"] = pd.cut(housing["median_income"], bins=[0., 1.5, 3.0, 4.5, 6., np.inf], labels=[1, 2, 3, 4, 5])
housing["income_cat"].hist()

In [ ]:
#stratified sampling based on the income category using Scikit-Learn’s StratifiedShuffleSplit class:
from sklearn.model_selection import StratifiedShuffleSplit
split = StratifiedShuffleSplit(n_splits=1, test_size=0.2, random_state=42)
for train_index, test_index in split.split(housing, housing["income_cat"]):
 strat_train_set = housing.loc[train_index]
 strat_test_set = housing.loc[test_index]
#look at income category proportions in the test set
print(strat_test_set["income_cat"].value_counts() / len(strat_test_set))

In [ ]:
#remove the income_cat attribute so the data is back to its original state
for set_ in (strat_train_set, strat_test_set):
 set_.drop("income_cat", axis=1, inplace=True)

# Discover and visualize the Data to Gain Insights
## Page 56
### This is only using the training set (put test data aside)

In [ ]:
#copy training set to play with
housing = strat_train_set.copy()

In [ ]:
#Visualize Geographical Data
#housing.plot(kind="scatter", x="longitude", y="latitude")
#set alpha option to visualize the places where there is a high density of data points
housing.plot(kind="scatter", x="longitude", y="latitude", alpha=0.1)

In [ ]:
#now look at housing prices
#each radius represents the district's population (s), color represents price (c)
#use predefined  color map (cmap)
housing.plot(kind="scatter", x="longitude", y="latitude", alpha=0.4, s=housing["population"]/100, label="population", figsize=(10,7), c="median_house_value", cmap=plt.get_cmap("jet"), colorbar=True)
plt.legend()

In [ ]:
#compute standard correlation coefficient (Pearson's r) between every pair of attributes using corr()
corr_matrix = housing.corr(numeric_only=True)
#look at how each attribute correlates with median house value
corr_matrix["median_house_value"].sort_values(ascending=False)

The correlation coefficient ranges from –1 to 1. When it is close to 1, it means that
there is a strong positive correlation; for example, the median house value tends to go
up when the median income goes up. When the coefficient is close to –1, it means
that there is a strong negative correlation; you can see a small negative correlation
between the latitude and the median house value (i.e., prices have a slight tendency to
go down when you go north). Finally, coefficients close to 0 mean that there is no
linear correlation.

In [ ]:
#another way to check for correlation between attributes is scatter_matrix()
#which plots every numerical attribute against every other numerical attribute
#limit to promising attributes
from pandas.plotting import scatter_matrix
attributes = ["median_house_value", "median_income", "total_rooms", "housing_median_age"]
scatter_matrix(housing[attributes], figsize=(12,8))

In [ ]:
#zoom in on the most promising attribute, median income
housing.plot(kind="scatter", x="median_income", y="median_house_value", alpha=0.1)

In [ ]:
#combine some attributes
housing["rooms_per_household"] = housing["total_rooms"]/housing["households"]
housing["bedrooms_per_room"] = housing["total_bedrooms"]/housing["total_rooms"]
housing["population_per_household"]=housing["population"]/housing["households"]
#look at the correlation matrix again:
corr_matrix = housing.corr(numeric_only=True)
corr_matrix["median_house_value"].sort_values(ascending=False)

# Prepare the Data for Machine Learning Algorithms
### Page 62

In [ ]:
#revert to a clean training set
housing = strat_train_set.drop("median_house_value", axis=1)
housing_labels = strat_train_set["median_house_value"].copy()

In [ ]:
#Data Cleaning
#Fix missing features since Machine Learning Algorithms cannot work with that
#use SimpleImputer to take care of missing values by replacing each missing value with the median of that attribute
from sklearn.impute import SimpleImputer
imputer = SimpleImputer(strategy="median")
#Since the median can only be computed on numerical attributes create a copy of the data without the text attribute ocean_proximity
housing_num = housing.drop("ocean_proximity", axis=1)
imputer.fit(housing_num)
print(imputer.statistics_)
print(housing_num.median().values)

In [ ]:
#now use the trained imputer to transform the training set by replacing missing values with learned medians
X = imputer.transform(housing_num)
#transofrm play NumPy array to DataFrame
housing_tr = pd.DataFrame(X, columns=housing_num.columns, index=housing_num.index)

In [ ]:
#Handling Text Attributes
housing_cat = housing[["ocean_proximity"]]
housing_cat.head(10)

In [ ]:
#Convert text to numbers for Machine Learning Algorithms using OrdinalEncoder class
from sklearn.preprocessing import OrdinalEncoder
ordinal_encoder = OrdinalEncoder()
housing_cat_encoded = ordinal_encoder.fit_transform(housing_cat)
housing_cat_encoded[:10]

In [ ]:
#get a list of categories using categories_ instance variable
from sklearn.preprocessing import OrdinalEncoder
ordinal_encoder.categories_

In [ ]:
# ML algorithms will assume that two nearby values are more similar than two distant values. This may be fine in some cases (e.g.,
#for ordered categories such as “bad,” “average,” “good,” and “excellent”), but it is obviously not the case for the ocean_proximity column
#(for example, categories 0 and 4are clearly more similar than categories 0 and 1). To fix this issue, a common solution is to create one binary
#attribute per category: one attribute equal to 1 when the category is “<1H OCEAN” (and 0 otherwise), another attribute equal to 1 when the cate‐
#gory is “INLAND” (and 0 otherwise), and so on. This is called one-hot encoding, because only one attribute will be equal to 1 (hot), while the
#others will be 0 (cold). The new attributes are sometimes called dummy attributes. Uses oneHotEncoder
from sklearn.preprocessing import OneHotEncoder
cat_encoder = OneHotEncoder()
housing_cat_1hot = cat_encoder.fit_transform(housing_cat)
housing_cat_1hot
#or to convert to a dense NumPy array
#housing_cat_1hot.toarray()
cat_encoder.categories_

# Custom Transformers

In [ ]:
#we will need to write our own for tasks such as custom cleanup or combining specific attributes.
#create a class and implement three methods, fit(), transform(), fit_transform()
from sklearn.base import BaseEstimator, TransformerMixin
rooms_ix, bedrooms_ix, population_ix, households_ix = 3, 4, 5, 6
class CombinedAttributesAdder(BaseEstimator, TransformerMixin):
 def __init__(self, add_bedrooms_per_room = True): # no *args or **kargs
     self.add_bedrooms_per_room = add_bedrooms_per_room
 def fit(self, X, y=None):
     return self # nothing else to do
 def transform(self, X):
     rooms_per_household = X[:, rooms_ix] / X[:, households_ix]
     population_per_household = X[:, population_ix] / X[:, households_ix]
     if self.add_bedrooms_per_room:
         bedrooms_per_room = X[:, bedrooms_ix] / X[:, rooms_ix]
         return np.c_[X, rooms_per_household, population_per_household, bedrooms_per_room]
     else:
        return np.c_[X, rooms_per_household, population_per_household]
attr_adder = CombinedAttributesAdder(add_bedrooms_per_room=False)
housing_extra_attribs = attr_adder.transform(housing.values)

## Feature Scaling
Not generally required. There are two common ways to get all attributes to have the same scale: min-max
scaling and standardizatio
## TransformationPipelinesn

In [ ]:
#many data transformation steps need to be executed in the right order, use Pipeline class to help with sequences
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import StandardScaler
num_pipeline = Pipeline([
 ('imputer', SimpleImputer(strategy="median")),
 ('attribs_adder', CombinedAttributesAdder()),
 ('std_scaler', StandardScaler()),
 ])
housing_num_tr = num_pipeline.fit_transform(housing_num)

In [ ]:
#apply all the transformations to the housing data
from sklearn.compose import ColumnTransformer
num_attribs = list(housing_num)
cat_attribs = ["ocean_proximity"]
full_pipeline = ColumnTransformer([
 ("num", num_pipeline, num_attribs),
 ("cat", OneHotEncoder(), cat_attribs),
 ])
housing_prepared = full_pipeline.fit_transform(housing)

# Select and Train a Model
## Training and Evaluating on the Training Set

In [ ]:
#first train a Linear Regression model

from sklearn.linear_model import LinearRegression
lin_reg = LinearRegression()
lin_reg.fit(housing_prepared, housing_labels)

In [ ]:
#try the Linear Regression model out on a few instances from training set
some_data = housing.iloc[:5]
some_labels = housing_labels.iloc[:5]
some_data_prepared = full_pipeline.transform(some_data)
print("Predictions:", lin_reg.predict(some_data_prepared))
print("Labels:", list(some_labels))
# predictions are not exactly accurate (e.g., the first prediction is off by close to 40%!).

In [ ]:
#Measure this regression model’s RMSE on the whole training set using mean_squared_error() function
from sklearn.metrics import mean_squared_error
housing_predictions = lin_reg.predict(housing_prepared)
lin_mse = mean_squared_error(housing_labels, housing_predictions)
lin_rmse = np.sqrt(lin_mse)
lin_rmse
#most districts’ median_housing_values range between $120,000 and $265,000, so a typical prediction error of
#$68,628 is not very satisfying. This is an example of a model underfitting the training data.

In [ ]:
#train a DecisionTreeRegressor. This is a powerful model, capable of finding complex nonlinear relationships in the data
from sklearn.tree import DecisionTreeRegressor
tree_reg = DecisionTreeRegressor()
tree_reg.fit(housing_prepared, housing_labels)


In [ ]:
#Now that the model is trained, let’s evaluate it on the training set:
housing_predictions = tree_reg.predict(housing_prepared)
tree_mse = mean_squared_error(housing_labels, housing_predictions)
tree_rmse = np.sqrt(tree_mse)
tree_rmse
#model too perfect (0.0), may have overfit the model now

In [ ]:
#use K-fold cross-validation feature. The following code randomly splits the training set into 10 distinct subsets called folds, then it
#trains and evaluates the Decision Tree model 10 times, picking a different fold for evaluation every time and training on the other 9 folds.
#The result is an array containing the 10 evaluation scores:
from sklearn.model_selection import cross_val_score
scores = cross_val_score(tree_reg, housing_prepared, housing_labels,
 scoring="neg_mean_squared_error", cv=10)
tree_rmse_scores = np.sqrt(-scores)
#print
def display_scores(scores):
    print("Scores:", scores)
    print("Mean:", scores.mean())
    print("Standard deviation:", scores.std())
display_scores(tree_rmse_scores)
#the Decision Tree model is overfitting so badly that it performs worse than the Linear Regression model.

In [ ]:
#try RandomForestRegressor model
#works by training many Decision Trees on random subsets of the features, then averaging out their predictions. Building a model on top of many
#other models is called Ensemble Learning, and it is often a great way to push ML algorithms even further
from sklearn.ensemble import RandomForestRegressor
forest_reg = RandomForestRegressor(n_estimators=10, random_state=42)
forest_reg.fit(housing_prepared, housing_labels)
housing_predictions = forest_reg.predict(housing_prepared)
forest_mse = mean_squared_error(housing_labels, housing_predictions)
forest_rmse = np.sqrt(forest_mse)
#print(display_scores(forest_rmse_scores))

In [ ]:
from sklearn.model_selection import cross_val_score

forest_scores = cross_val_score(forest_reg, housing_prepared, housing_labels,
                                scoring="neg_mean_squared_error", cv=10)
forest_rmse_scores = np.sqrt(-forest_scores)
print(display_scores(forest_rmse_scores))
#much better than the rest but still overfitting

# Chapter 2, Exercise 1
Try a Support Vector Machine regressor (sklearn.svm.SVR) with various hyperparameters, such as kernel="linear" (with various values for the C
hyperparameter) or kernel="rbf" (with various values for the C and gamma hyperparameters). Don’t worry about what these hyperparameters mean for now.
How does the best SVR predictor perform?

### Grid Search

# Chapter 2, Exercise 2
Try replacing GridSearchCV with RandomizedSearchCV.

# Chapter 2, Exercise 3
Try adding a transformer in the preparation pipeline to select only the most important attributes.

# Chapter 2, Exercise 4
Try creating a single pipeline that does the full data preparation plus the final prediction

# Chapter 2, Exercise 5
Automatically explore some preparation options using GridSearchCV.

# Chapter 3
## Using MNIST dataset

In [ ]:
# Import the function to fetch datasets from OpenML, an online repository of well-documented datasets
from sklearn.datasets import fetch_openml

# Fetch MNIST dataset containing handwritten digits with 784 features
mnist = fetch_openml('mnist_784', version=1,as_frame=False)

# keys of the MNIST dataset. object contains the data, target (labels), a description of the dataset, and other metadata.
# keys: 'data': the feature matrix,'target': the label array,'feature_names': the names of the features,'DESCR': a full description of the dataset,
# 'categories': the category labels (for categorical features)
mnist_keys = mnist.keys()

In [ ]:
# examine the structure of the data (X) and labels (y)
X, y = mnist["data"], mnist["target"]
#return the dimensions of the array containing the image data
X.shape
#return the dimensions of the array containing the labels for the images
y.shape

In [ ]:
import matplotlib.pyplot as plt
import numpy as np
#an instance’s feature vector, reshape it to a 28 × 28 array, and display it using Matplotlib’s imshow() function:
some_digit = X[0]
some_digit_image = some_digit.reshape(28, 28)
plt.imshow(some_digit_image, cmap="binary")
plt.axis("off")
plt.show()


In [ ]:
#double check the label to make sure it is what we think it is
y[0]
#cast the string label to an int since that is what most ML algorithms expect
y = y.astype(np.uint8)

In [ ]:
#create a test set first (for this datsset using already given split)
X_train, X_test, y_train, y_test = X[:60000], X[60000:], y[:60000], y[60000:]

## Training a Binary Classifier
Distinguishes between two classes

In [ ]:
#target vectors for this classification task
y_train_5 = (y_train == 5) # True for all 5s, False for all other digits
y_test_5 = (y_test == 5)

In [ ]:
# pick a classifier and train it
# stochastic gradient descent classifier, train it to differentiate between the digit '5' and other digits in the MNIST dataset
#The X_train contains the training data features, while y_train_5 is the target variable for the binary classification task
from sklearn.linear_model import SGDClassifier
#random_state gives you reproducible results
sgd_clf = SGDClassifier(random_state=42)
sgd_clf.fit(X_train, y_train_5)
#detect images of the number 5
sgd_clf.predict([some_digit])

## Performance Measurements

In [ ]:
#meausry accuracy Using Cross-Validation
#evaluates the model
from sklearn.model_selection import StratifiedKFold
from sklearn.base import clone
skfolds = StratifiedKFold(n_splits=3, shuffle=True, random_state=42)
for train_index, test_index in skfolds.split(X_train, y_train_5):
 clone_clf = clone(sgd_clf)
 X_train_folds = X_train[train_index]
 y_train_folds = y_train_5[train_index]
 X_test_fold = X_train[test_index]
 y_test_fold = y_train_5[test_index]
 clone_clf.fit(X_train_folds, y_train_folds)
 y_pred = clone_clf.predict(X_test_fold)
 n_correct = sum(y_pred == y_test_fold)
 print(n_correct / len(y_pred))

In [ ]:
#Use cross_val_score() to evaluate SGDClassifier. using K-fold cross-validation with three folds, ie splitting the training set into K folds
#then making predictions and evaluating them on each fold using a model trained on the remaining folds
from sklearn.model_selection import cross_val_score
cross_val_score(sgd_clf, X_train, y_train_5, cv=3, scoring="accuracy")
# Above 93% accuracy

In [ ]:
#very dumb classifier that just classifies every single image in the “not-5” class:
from sklearn.base import BaseEstimator
class Never5Classifier(BaseEstimator):
 def fit(self, X, y=None):
     return self
 def predict(self, X):
     return np.zeros((len(X), 1), dtype=bool)

never_5_clf = Never5Classifier()
cross_val_score(never_5_clf, X_train, y_train_5, cv=3, scoring="accuracy")
#90% accuracy bc about 10% of the images are 5, so not too hard to do

In [ ]:
#Great example as to why accuracy is not the preferred performance measure for classifiers
#A better way is a Confusion Matrix
#like the cross_val_score() function, cross_val_predict() performs K-fold cross-validation, but instead of returning the evaluation scores,
#it returns the predictions made on each test fold.
from sklearn.model_selection import cross_val_predict
y_train_pred = cross_val_predict(sgd_clf, X_train, y_train_5, cv=3)
# get the confusion matrix using the confusion_matrix() function. pass it the target classes (y_train_5) and the predicted classes (y_train_pred):
from sklearn.metrics import confusion_matrix
confusion_matrix(y_train_5, y_train_pred)
#Each row in a confusion matrix represents an actual class, while each column represents a predicted class.

In [ ]:
#comute classifier metrics, measurements used to evaluate the performance of a classification model
#compute the harmonic mean of precision and recall/(F_1)
from sklearn.metrics import f1_score
f1_score(y_train_5, y_train_pred)

In [ ]:
#this decision_function method returns the decision score for each instance
y_scores = sgd_clf.decision_function([some_digit])
y_scores
threshold = 0
y_some_digit_pred = (y_scores > threshold)

In [ ]:
#The SGDClassifier uses a threshold equal to 0, so the previous code returns the same result as the predict() method (i.e., True).
#raise the threshold:
threshold = 8000
y_some_digit_pred = (y_scores > threshold)
y_some_digit_pred
#raising threshold reduces recall and classifier no longer detects it as a 5 as it did when threshold = 0

In [ ]:
#decdide what threshold to use:
#use cross_val_predict() function to get the scores of all instances in the training set,
#specify that you want to return decision scores instead of predictions:
y_scores = cross_val_predict(sgd_clf, X_train, y_train_5, cv=3, method="decision_function")

In [ ]:
#Then with these scores, use the precision_recall_curve() function to compute precision and recall for all possible thresholds:
from sklearn.metrics import precision_recall_curve
precisions, recalls, thresholds = precision_recall_curve(y_train_5, y_scores)

In [ ]:
# Matplotlib to plot precision and recall as functions of the threshold value
def plot_precision_recall_vs_threshold(precisions, recalls, thresholds):
    plt.plot(thresholds, precisions[:-1], "b--", label="Precision", linewidth=2)
    plt.plot(thresholds, recalls[:-1], "g-", label="Recall", linewidth=2)
    plt.legend(loc="center right", fontsize=16)
    plt.xlabel("Threshold", fontsize=16)
    plt.grid(True)
    plt.axis([-50000, 50000, 0, 1])

recall_90_precision = recalls[np.argmax(precisions >= 0.90)]
threshold_90_precision = thresholds[np.argmax(precisions >= 0.90)]

plt.figure(figsize=(8, 4))
plot_precision_recall_vs_threshold(precisions, recalls, thresholds)
plt.plot([threshold_90_precision, threshold_90_precision], [0., 0.9], "r:")
plt.plot([-50000, threshold_90_precision], [0.9, 0.9], "r:")
plt.plot([-50000, threshold_90_precision], [recall_90_precision, recall_90_precision], "r:")
plt.plot([threshold_90_precision], [0.9], "ro")
plt.plot([threshold_90_precision], [recall_90_precision], "ro")
plt.show()


#     [...] # highlight the threshold and add the legend, axis label, and grid
#     # Highlight the threshold (e.g., with a red vertical line at the chosen threshold value)
#     chosen_threshold = 5000
#     plt.plot([chosen_threshold, chosen_threshold], [0, 1], "r:")     # Threshold line
#     plt.annotate('Threshold', xy=(chosen_threshold, 0.5), xytext=(chosen_threshold+500, 0.5), arrowprops=dict(facecolor='black', shrink=0.05), )
#     plt.xlabel("Threshold")                                           # x-axis label
#     plt.ylabel("Score")                                               # y-axis label
#     plt.legend()                                                      # Show legend
#     plt.grid(True)                                                    # Turn on the grid
#     plt.xlim([-50000, 50000])

# plot_precision_recall_vs_threshold(precisions, recalls, thresholds)
# plt.show()

In [ ]:
(y_train_pred == (y_scores > 0)).all()

In [ ]:
def plot_precision_vs_recall(precisions, recalls):
    plt.plot(recalls, precisions, "b-", linewidth=2)
    plt.xlabel("Recall", fontsize=16)
    plt.ylabel("Precision", fontsize=16)
    plt.axis([0, 1, 0, 1])
    plt.grid(True)

plt.figure(figsize=(8, 6))
plot_precision_vs_recall(precisions, recalls)
plt.plot([recall_90_precision, recall_90_precision], [0., 0.9], "r:")
plt.plot([0.0, recall_90_precision], [0.9, 0.9], "r:")
plt.plot([recall_90_precision], [0.9], "ro")
plt.show()

In [ ]:
#if we want a 90% precision
threshold_90_precision = thresholds[np.argmax(precisions >= 0.90)]

In [ ]:
threshold_90_precision

In [ ]:
#to make predicitions instead of calling classifier's predicti() method
y_train_pred_90 = (y_scores >= threshold_90_precision)

In [ ]:
#check precision
precision_score(y_train_5, y_train_pred_90)
#this confirms 90% precision we wanted

In [ ]:
#check recall
recall_score(y_train_5, y_train_pred_90)

## The ROC Curve
Receiver Operating Characteristic. Another classifier metric used with binary classifiers. Plots TPR (recall) against FPR.

In [ ]:
#Compute TPR and FPR
from sklearn.metrics import roc_curve

fpr, tpr, thresholds = roc_curve(y_train_5, y_scores)

In [ ]:
#then plot TPR and FPR
def plot_roc_curve(fpr, tpr, label=None):
    plt.plot(fpr, tpr, linewidth=2, label=label)
    plt.plot([0, 1], [0, 1], 'k--') # dashed diagonal
    plt.axis([0, 1, 0, 1])
    plt.xlabel('False Positive Rate (Fall-Out)', fontsize=16)
    plt.ylabel('True Positive Rate (Recall)', fontsize=16)
    plt.grid(True)

plt.figure(figsize=(8, 6))
plot_roc_curve(fpr, tpr)
fpr_90 = fpr[np.argmax(tpr >= recall_90_precision)]
plt.plot([fpr_90, fpr_90], [0., recall_90_precision], "r:")
plt.plot([0.0, fpr_90], [recall_90_precision, recall_90_precision], "r:")
plt.plot([fpr_90], [recall_90_precision], "ro")
plt.show()

In [ ]:
#The more your ROC line is above the randoming guessing line the better. Use AUC to calculate this. 0.5 is purely random, 1 is perfect
from sklearn.metrics import roc_auc_score

roc_auc_score(y_train_5, y_scores)

In [ ]:
#Train RandomForestClassifer and compare it's ROC curve and ROC AUC score to the SGD Classifier
#The predict_proba() method returns an array containing a row per instance and a column per class, each containing the probability that the given instance belongs to the given class (e.g., 70% chance that the image represents a 5):

from sklearn.ensemble import RandomForestClassifier
forest_clf = RandomForestClassifier(n_estimators=100, random_state=42)
y_probas_forest = cross_val_predict(forest_clf, X_train, y_train_5, cv=3,
                                    method="predict_proba")

In [ ]:
#roc_curve() function expects labels and scores, but instead of scores you can give it class probabilities. Let’s use the positive class’s probability as the score:
y_scores_forest = y_probas_forest[:, 1] # score = proba of positive class
fpr_forest, tpr_forest, thresholds_forest = roc_curve(y_train_5,y_scores_forest)

In [ ]:
#plot ROC curve  to see how they compare
recall_for_forest = tpr_forest[np.argmax(fpr_forest >= fpr_90)]

plt.figure(figsize=(8, 6))
plt.plot(fpr, tpr, "b:", linewidth=2, label="SGD")
plot_roc_curve(fpr_forest, tpr_forest, "Random Forest")
plt.plot([fpr_90, fpr_90], [0., recall_90_precision], "r:")
plt.plot([0.0, fpr_90], [recall_90_precision, recall_90_precision], "r:")
plt.plot([fpr_90], [recall_90_precision], "ro")
plt.plot([fpr_90, fpr_90], [0., recall_for_forest], "r:")
plt.plot([fpr_90], [recall_for_forest], "ro")
plt.grid(True)
plt.legend(loc="lower right", fontsize=16)
plt.show()

In [ ]:
#RandomForest is better bc ROC Curve looks much better than SGDClassifier's as it comes much closer to top-left corner. As a result its ROC AUC score is also signifigantly better
roc_auc_score(y_train_5, y_scores_forest)
#99% precision

In [ ]:
y_train_pred_forest = cross_val_predict(forest_clf, X_train, y_train_5, cv=3)
precision_score(y_train_5, y_train_pred_forest)

In [ ]:
recall_score(y_train_5, y_train_pred_forest)

In [ ]:
## Multiclass Classification

In [ ]:
#Run a Support Vector Machine CLassifier (this algorithm is strictly binary)
#trains the SVC on the training set using the 0-9 target classes, not 5 versus the rest (y_train_5)
from sklearn.svm import SVC

svm_clf = SVC(gamma="auto", random_state=42)
svm_clf.fit(X_train[:1000], y_train[:1000]) # y_train, not y_train_5
svm_clf.predict([some_digit])
#this actually trained 45 binary classifiers,got their deicision scores for the image, and selected teh class that won the most duels.

In [ ]:
#shown by returning the scores per instance
some_digit_scores = svm_clf.decision_function([some_digit])
some_digit_scores

In [ ]:
#highest score is one correspeonding to class 5
np.argmax(some_digit_scores)
svm_clf.classes_
svm_clf.classes_[5]

In [ ]:
#to force skikit to use a certain classifier (OvO, OvR, etc
#this creates a multiclass classifier using the OvR strategy based on SVR
from sklearn.multiclass import OneVsRestClassifier
ovr_clf = OneVsRestClassifier(SVC(gamma="auto", random_state=42))
ovr_clf.fit(X_train[:1000], y_train[:1000])
ovr_clf.predict([some_digit])
len(ovr_clf.estimators_)

In [ ]:
#Or using the SGDClassifier
sgd_clf.fit(X_train, y_train)
sgd_clf.predict([some_digit])

In [ ]:
#since Skikit used the OvR strategy, there are now 10 classes and it trained 10 binary classifiers. THis now returns one value per class
#score that SGD Classifier assigned to each class
sgd_clf.decision_function([some_digit])

In [ ]:
cross_val_score(sgd_clf, X_train, y_train, cv=3, scoring="accuracy")

In [ ]:
from sklearn.preprocessing import StandardScaler
scaler = StandardScaler()
X_train_scaled = scaler.fit_transform(X_train.astype(np.float64))
cross_val_score(sgd_clf, X_train_scaled, y_train, cv=3, scoring="accuracy")

## Error Analysis

In [ ]:
y_train_pred = cross_val_predict(sgd_clf, X_train_scaled, y_train, cv=3)
conf_mx = confusion_matrix(y_train, y_train_pred)
conf_mx

In [ ]:
# since sklearn 0.22, you can use sklearn.metrics.plot_confusion_matrix()
def plot_confusion_matrix(matrix):
    """If you prefer color and a colorbar"""
    fig = plt.figure(figsize=(8,8))
    ax = fig.add_subplot(111)
    cax = ax.matshow(matrix)
    fig.colorbar(cax)

In [ ]:
plt.matshow(conf_mx, cmap=plt.cm.gray)
save_fig("confusion_matrix_plot", tight_layout=False)
plt.show()

In [ ]:
row_sums = conf_mx.sum(axis=1, keepdims=True)
norm_conf_mx = conf_mx / row_sums

In [ ]:
np.fill_diagonal(norm_conf_mx, 0)
plt.matshow(norm_conf_mx, cmap=plt.cm.gray)
save_fig("confusion_matrix_errors_plot", tight_layout=False)
plt.show()

In [ ]:
cl_a, cl_b = 3, 5
X_aa = X_train[(y_train == cl_a) & (y_train_pred == cl_a)]
X_ab = X_train[(y_train == cl_a) & (y_train_pred == cl_b)]
X_ba = X_train[(y_train == cl_b) & (y_train_pred == cl_a)]
X_bb = X_train[(y_train == cl_b) & (y_train_pred == cl_b)]

plt.figure(figsize=(8,8))
plt.subplot(221); plot_digits(X_aa[:25], images_per_row=5)
plt.subplot(222); plot_digits(X_ab[:25], images_per_row=5)
plt.subplot(223); plot_digits(X_ba[:25], images_per_row=5)
plt.subplot(224); plot_digits(X_bb[:25], images_per_row=5)
save_fig("error_analysis_digits_plot")
plt.show()

## Multilabel Classification

In [ ]:
from sklearn.neighbors import KNeighborsClassifier

y_train_large = (y_train >= 7)
y_train_odd = (y_train % 2 == 1)
y_multilabel = np.c_[y_train_large, y_train_odd]

knn_clf = KNeighborsClassifier()
knn_clf.fit(X_train, y_multilabel)

In [ ]:
knn_clf.predict([some_digit])

In [ ]:
y_train_knn_pred = cross_val_predict(knn_clf, X_train, y_multilabel, cv=3)
f1_score(y_multilabel, y_train_knn_pred, average="macro")

## Multioutput Classification

In [ ]:
noise = np.random.randint(0, 100, (len(X_train), 784))
X_train_mod = X_train + noise
noise = np.random.randint(0, 100, (len(X_test), 784))
X_test_mod = X_test + noise
y_train_mod = X_train
y_test_mod = X_test

In [ ]:
some_index = 0
plt.subplot(121); plot_digit(X_test_mod[some_index])
plt.subplot(122); plot_digit(y_test_mod[some_index])
save_fig("noisy_digit_example_plot")
plt.show()

In [ ]:
knn_clf.fit(X_train_mod, y_train_mod)
clean_digit = knn_clf.predict([X_test_mod[some_index]])
plot_digit(clean_digit)
save_fig("cleaned_digit_example_plot")

# Chapter 3, Exercise 1
 Try to build a classifier for the MNIST dataset that achieves over 97% accuracy
on the test set. Hint: the KNeighborsClassifier works quite well for this task;
you just need to find good hyperparameter values (try a grid search on the
weights and n_neighbors hyperparameters).

In [ ]:
#GridSearchCV will use cross-validation to evaluate all possible combinations of hyperparameters so you don't have to tinker
# Import the GridSearchCV class for hyperparameter tuning using cross-validation.
from sklearn.model_selection import GridSearchCV

# Define a parameter grid to search over. For the KNeighborsClassifier, we're exploring
# two hyperparameters: 'weights' (with options 'uniform' and 'distance') and 'n_neighbors'
# (with options 3, 4, and 5). This creates a grid of parameter combinations to test.
param_grid = [{'weights': ["uniform", "distance"], 'n_neighbors': [3, 4, 5]}]

# Initialize Classifier. This is the model we're tuning.
knn_clf = KNeighborsClassifier()
# Set up GridSearchCV with the classifier (knn_clf), the parameter grid (param_grid),
# and a 5-fold cross-validation (cv=5). Verbose=3 increases the messages printed
# to the console so you can track the progress of the search.
grid_search = GridSearchCV(knn_clf, param_grid, cv=5, verbose=3)
# Fit the GridSearchCV to the training data. This will test all combinations
# of parameters in the grid, using 5-fold cross-validation for each combination.
# It selects the best combination based on cross-validated performance.
grid_search.fit(X_train, y_train)
# After fitting, grid_search holds the best model and parameters
grid_search_clf.best_score_
#best parameters
grid_search_clf.best_params_
#best model
grid_search.best_estimator_

In [ ]:
# Import the function to compute accuracy score
from sklearn.metrics import accuracy_score

# Use the best model found by GridSearchCV to make predictions on the test set.
# This uses the model with the optimal hyperparameters found during the grid search.
y_pred = grid_search.predict(X_test)
# Calculate the accuracy of the model on the test set by comparing the predicted labels (y_pred)
# to the true labels (y_test). The accuracy score is the fraction of correct predictions over
# the total number of predictions, expressed as a float between 0 and 1, where 1 means perfect accuracy.
accuracy_score(y_test, y_pred)
# The variable 'accuracy' now holds the accuracy score of the best model found by GridSearchCV
# when evaluated on unseen test data. This gives an estimate of the model's generalization ability.

# Chapter 3, Exercise 2
 Write a function that can shift an MNIST image in any direction (left, right, up,
or down) by one pixel.5 Then, for each image in the training set, create four shif‐
ted copies (one per direction) and add them to the training set. Finally, train your
best model on this expanded training set and measure its accuracy on the test set.
You should observe that your model performs even better now! This technique of
artificially growing the training set is called data augmentation or training set
expansion.

In [ ]:
# Define a function to shift an image by dx (delta x) and dy (delta y) pixels.
# This can be used for data augmentation, to create more training data from the existing images
# by slightly shifting them in any direction.
def shift_image(image, dx, dy):
    # Reshape the flat image array into a 28x28 matrix, as the original MNIST images are 28x28 pixels.
    image = image.reshape((28, 28))
    # Use the 'shift' function from scipy.ndimage.interpolation to shift the image.
    # 'dy' and 'dx' specify the shift amount along the y and x axes, respectively.
    # 'cval' specifies the value to fill past edges of input if mode is 'constant'. Here it's set to 0,
    # meaning the empty space created by the shift will be filled with 0s (black).
    # 'mode' specifies how the input array is extended beyond its boundaries. 'constant' means pad with a constant value.
    shifted_image = shift(image, [dy, dx], cval=0, mode="constant")
    # Reshape the shifted image back to a flat array before returning it.
    return shifted_image.reshape([-1])


In [ ]:
image = X_train[1000]
shifted_image_down = shift_image(image, 0, 5)
shifted_image_left = shift_image(image, -5, 0)

plt.figure(figsize=(12,3))
plt.subplot(131)
plt.title("Original", fontsize=14)
plt.imshow(image.reshape(28, 28), interpolation="nearest", cmap="Greys")
plt.subplot(132)
plt.title("Shifted down", fontsize=14)
plt.imshow(shifted_image_down.reshape(28, 28), interpolation="nearest", cmap="Greys")
plt.subplot(133)
plt.title("Shifted left", fontsize=14)
plt.imshow(shifted_image_left.reshape(28, 28), interpolation="nearest", cmap="Greys")
plt.show()

In [ ]:
# Initialize lists to hold the augmented training data and labels.
# Copy original training images.
X_train_augmented = [image for image in X_train]
# Copy original training labels.
y_train_augmented = [label for label in y_train]

# Loop over a set of directions to shift the images: right (1, 0), left (-1, 0), down (0, 1), and up (0, -1).
for dx, dy in ((1, 0), (-1, 0), (0, 1), (0, -1)):
    # For each direction, shift each image in the training set and add the shifted image to the augmented dataset.

    for image, label in zip(X_train, y_train):
        # Shift the image using the previously defined function.
        shifted_image = shift_image(image, dx, dy)
# Add the shifted image to the augmented dataset.
        X_train_augmented.append(shift_image(image, dx, dy))
        # The label remains the same, as the image content hasn't changed category.
        y_train_augmented.append(label)
# Convert the augmented datasets from lists to numpy arrays for easier handling in machine learning models.
X_train_augmented = np.array(X_train_augmented)
y_train_augmented = np.array(y_train_augmented)


In [ ]:
import numpy as np

# Generate a random permutation of indices based on the length of the augmented training set.
shuffle_idx = np.random.permutation(len(X_train_augmented))

# Reorder the augmented training images and labels according to the random permutation.
# This ensures that the data is shuffled, mixing the original and augmented images,
# which is beneficial for training models to prevent any order bias.
X_train_augmented = X_train_augmented[shuffle_idx]
y_train_augmented = y_train_augmented[shuffle_idx]


In [ ]:
knn_clf = KNeighborsClassifier(**grid_search.best_params_)
knn_clf.fit(X_train_augmented, y_train_augmented)

In [ ]:
y_pred = knn_clf.predict(X_test)
accuracy_score(y_test, y_pred)

# Chapter 3, Exercise 3
 Tackle the Titanic dataset. A great place to start is on Kaggle (https://www.kaggle.com/c/titanic).

In [ ]:
import os
import urllib.request

TITANIC_PATH = os.path.join("datasets", "titanic")
DOWNLOAD_URL = "https://raw.githubusercontent.com/ageron/handson-ml2/master/datasets/titanic/"

def fetch_titanic_data(url=DOWNLOAD_URL, path=TITANIC_PATH):
    if not os.path.isdir(path):
        os.makedirs(path)
    for filename in ("train.csv", "test.csv"):
        filepath = os.path.join(path, filename)
        if not os.path.isfile(filepath):
            print("Downloading", filename)
            urllib.request.urlretrieve(url + filename, filepath)

fetch_titanic_data()

In [ ]:
import pandas as pd

def load_titanic_data(filename, titanic_path=TITANIC_PATH):
    csv_path = os.path.join(titanic_path, filename)
    return pd.read_csv(csv_path)

In [ ]:
train_data = load_titanic_data("train.csv")
test_data = load_titanic_data("test.csv")

In [ ]:
#look at attributes
train_data.head()

In [ ]:
#set PassengerId as index column
train_data = train_data.set_index("PassengerId")
test_data = test_data.set_index("PassengerId")

In [ ]:
#get more info - any missing data?
train_data.info()

In [ ]:
#median age of females
train_data[train_data["Sex"]=="female"]["Age"].median()
#to replace null Age attributes with median age

In [ ]:
#look at numerical attributes
train_data.describe()

In [ ]:
#check in boolean
train_data["Survived"].value_counts()

In [ ]:
#look at categorical attributes
train_data["Pclass"].value_counts()

In [ ]:
train_data["Sex"].value_counts()

In [ ]:
train_data["Embarked"].value_counts()

In [ ]:
#build our preprocessing pipelines, starting with the pipeline for numerical attributes:
from sklearn.pipeline import Pipeline
from sklearn.impute import SimpleImputer
from sklearn.preprocessing import StandardScaler

num_pipeline = Pipeline([
        ("imputer", SimpleImputer(strategy="median")),
        ("scaler", StandardScaler())
    ])

In [ ]:
#build the pipeline for the categorical attributes:
from sklearn.preprocessing import OneHotEncoder
cat_pipeline = Pipeline([
        ("imputer", SimpleImputer(strategy="most_frequent")),
        ("cat_encoder", OneHotEncoder(sparse=False)),
    ])

In [ ]:
#join the two
from sklearn.compose import ColumnTransformer

num_attribs = ["Age", "SibSp", "Parch", "Fare"]
cat_attribs = ["Pclass", "Sex", "Embarked"]

preprocess_pipeline = ColumnTransformer([
        ("num", num_pipeline, num_attribs),
        ("cat", cat_pipeline, cat_attribs),
    ])

In [ ]:
X_train = preprocess_pipeline.fit_transform(
    train_data[num_attribs + cat_attribs])
X_train

In [ ]:
#get labels
y_train = train_data["Survived"]

In [ ]:
#train classifier
from sklearn.ensemble import RandomForestClassifier

forest_clf = RandomForestClassifier(n_estimators=100, random_state=42)
forest_clf.fit(X_train, y_train)

In [ ]:
#make predictions
X_test = preprocess_pipeline.transform(test_data[num_attribs + cat_attribs])
y_pred = forest_clf.predict(X_test)

In [ ]:
#use cross validation to determine how good our model is
from sklearn.model_selection import cross_val_score

forest_scores = cross_val_score(forest_clf, X_train, y_train, cv=10)
forest_scores.mean()

In [ ]:
#try to get more accurate classifier with SvC
from sklearn.svm import SVC

svm_clf = SVC(gamma="auto")
svm_scores = cross_val_score(svm_clf, X_train, y_train, cv=10)
svm_scores.mean()

In [ ]:
#plot the ten scores
import matplotlib.pyplot as plt

plt.figure(figsize=(8, 4))
plt.plot([1]*10, svm_scores, ".")
plt.plot([2]*10, forest_scores, ".")
plt.boxplot([svm_scores, forest_scores], labels=("SVM","Random Forest"))
plt.ylabel("Accuracy", fontsize=14)
plt.show()
#can further improve

In [ ]:
train_data["AgeBucket"] = train_data["Age"] // 15 * 15
train_data[["AgeBucket", "Survived"]].groupby(['AgeBucket']).mean()

In [ ]:
train_data["RelativesOnboard"] = train_data["SibSp"] + train_data["Parch"]
train_data[["RelativesOnboard", "Survived"]].groupby(['RelativesOnboard']).mean()

# Chapter 3, Exercise 4
Build a spam classifier (a more challenging exercise):

• Download examples of spam and ham from Apache SpamAssassin’s public
dataset (https://homl.info/spamassassin)..

• Unzip the datasets and familiarize yourself with the data format.

• Split the datasets into a training set and a test set.

• Write a data preparation pipeline to convert each email into a feature vector.
Your preparation pipeline should transform an email into a (sparse) vector that
indicates the presence or absence of each possible word. For example, if all
emails only ever contain four words, “Hello,” “how,” “are,” “you,” then the email
“Hello you Hello Hello you” would be converted into a vector [1, 0, 0, 1]
(meaning [“Hello” is present, “how” is absent, “are” is absent, “you” is
present]), or [3, 0, 0, 2] if you prefer to count the number of occurrences of
each word.

You may want to add hyperparameters to your preparation pipeline to control
whether or not to strip off email headers, convert each email to lowercase,
remove punctuation, replace all URLs with “URL,” replace all numbers with
“NUMBER,” or even perform stemming (i.e., trim off word endings; there are
Python libraries available to do this).
Finally, try out several classifiers and see if you can build a great spam classi‐
fier, with both high recall and high precision.


In [ ]:
#retrive data
import os
import tarfile
import urllib.request

DOWNLOAD_ROOT = "http://spamassassin.apache.org/old/publiccorpus/"
HAM_URL = DOWNLOAD_ROOT + "20030228_easy_ham.tar.bz2"
SPAM_URL = DOWNLOAD_ROOT + "20030228_spam.tar.bz2"
SPAM_PATH = os.path.join("datasets", "spam")

def fetch_spam_data(ham_url=HAM_URL, spam_url=SPAM_URL, spam_path=SPAM_PATH):
    if not os.path.isdir(spam_path):
        os.makedirs(spam_path)
    for filename, url in (("ham.tar.bz2", ham_url), ("spam.tar.bz2", spam_url)):
        path = os.path.join(spam_path, filename)
        if not os.path.isfile(path):
            urllib.request.urlretrieve(url, path)
        tar_bz2_file = tarfile.open(path)
        tar_bz2_file.extractall(path=spam_path)
        tar_bz2_file.close()

fetch_spam_data()

In [ ]:
#download emails
HAM_DIR = os.path.join(SPAM_PATH, "easy_ham")
SPAM_DIR = os.path.join(SPAM_PATH, "spam")
ham_filenames = [name for name in sorted(os.listdir(HAM_DIR)) if len(name) > 20]
spam_filenames = [name for name in sorted(os.listdir(SPAM_DIR)) if len(name) > 20]

In [ ]:
len(ham_filenames)

In [ ]:
len(spam_filenames)

In [ ]:
#use Python's email module to parse these emails (this handles headers, encoding, etc
import email
import email.policy

def load_email(is_spam, filename, spam_path=SPAM_PATH):
    directory = "spam" if is_spam else "easy_ham"
    with open(os.path.join(spam_path, directory, filename), "rb") as f:
        return email.parser.BytesParser(policy=email.policy.default).parse(f)

In [ ]:
ham_emails = [load_email(is_spam=False, filename=name) for name in ham_filenames]
spam_emails = [load_email(is_spam=True, filename=name) for name in spam_filenames]

In [ ]:
#look at one example of ham and one example of spam, to get a feel of what the data looks like:
print(ham_emails[1].get_content().strip())

In [ ]:
#same for spam emails
print(spam_emails[6].get_content().strip())

In [ ]:
#Some emails are actually multipart, with images and attachments. Look at the various types of structures we have:
def get_email_structure(email):
    if isinstance(email, str):
        return email
    payload = email.get_payload()
    if isinstance(payload, list):
        return "multipart({})".format(", ".join([
            get_email_structure(sub_email)
            for sub_email in payload
        ]))
    else:
        return email.get_content_type()

In [ ]:
from collections import Counter

def structures_counter(emails):
    structures = Counter()
    for email in emails:
        structure = get_email_structure(email)
        structures[structure] += 1
    return structures

In [ ]:
structures_counter(ham_emails).most_common()

In [ ]:
structures_counter(spam_emails).most_common()

In [ ]:
#look at the email headers:
for header, value in spam_emails[0].items():
    print(header,":",value)

In [ ]:
#focus on the Subject header:
spam_emails[0]["Subject"]

In [ ]:
#split it into a training set and a test set:
import numpy as np
from sklearn.model_selection import train_test_split

X = np.array(ham_emails + spam_emails, dtype=object)
y = np.array([0] * len(ham_emails) + [1] * len(spam_emails))

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

In [ ]:
#start writing the preprocessing functions. First,  a function to convert HTML to plain text.
#The following function first drops the <head> section, then converts all <a> tags to the word HYPERLINK, then it gets rid of all HTML tags, leaving only the plain text. For readability, it also replaces multiple newlines with single newlines, and finally it unescapes html entities (such as &gt; or &nbsp;):
import re
from html import unescape

def html_to_plain_text(html):
    text = re.sub('<head.*?>.*?</head>', '', html, flags=re.M | re.S | re.I)
    text = re.sub('<a\s.*?>', ' HYPERLINK ', text, flags=re.M | re.S | re.I)
    text = re.sub('<.*?>', '', text, flags=re.M | re.S)
    text = re.sub(r'(\s*\n)+', '\n', text, flags=re.M | re.S)
    return unescape(text)

In [ ]:
#HTML spam
html_spam_emails = [email for email in X_train[y_train==1]
                    if get_email_structure(email) == "text/html"]
sample_html_spam = html_spam_emails[7]
print(sample_html_spam.get_content().strip()[:1000], "...")

In [ ]:
#resulting plain text
print(html_to_plain_text(sample_html_spam.get_content())[:1000], "...")

In [ ]:
#function that takes an email as input and returns its content as plain text, no matter its format is:
def email_to_text(email):
    html = None
    for part in email.walk():
        ctype = part.get_content_type()
        if not ctype in ("text/plain", "text/html"):
            continue
        try:
            content = part.get_content()
        except: # in case of encoding issues
            content = str(part.get_payload())
        if ctype == "text/plain":
            return content
        else:
            html = content
    if html:
        return html_to_plain_text(html)

In [ ]:
print(email_to_text(sample_html_spam)[:100], "...")

In [ ]:
#stemming
try:
    import nltk

    stemmer = nltk.PorterStemmer()
    for word in ("Computations", "Computation", "Computing", "Computed", "Compute", "Compulsive"):
        print(word, "=>", stemmer.stem(word))
except ImportError:
    print("Error: stemming requires the NLTK module.")
    stemmer = None

In [ ]:
#way to replace URLs with the word "URL"
%pip install -q -U urlextract

In [ ]:
try:
    import urlextract # may require an Internet connection to download root domain names

    url_extractor = urlextract.URLExtract()
    print(url_extractor.find_urls("Will it detect github.com and https://youtu.be/7Pq-S557XQU?t=3m32s"))
except ImportError:
    print("Error: replacing URLs requires the urlextract module.")
    url_extractor = None

In [ ]:
#transformer
from sklearn.base import BaseEstimator, TransformerMixin

class EmailToWordCounterTransformer(BaseEstimator, TransformerMixin):
    def __init__(self, strip_headers=True, lower_case=True, remove_punctuation=True,
                 replace_urls=True, replace_numbers=True, stemming=True):
        self.strip_headers = strip_headers
        self.lower_case = lower_case
        self.remove_punctuation = remove_punctuation
        self.replace_urls = replace_urls
        self.replace_numbers = replace_numbers
        self.stemming = stemming
    def fit(self, X, y=None):
        return self
    def transform(self, X, y=None):
        X_transformed = []
        for email in X:
            text = email_to_text(email) or ""
            if self.lower_case:
                text = text.lower()
            if self.replace_urls and url_extractor is not None:
                urls = list(set(url_extractor.find_urls(text)))
                urls.sort(key=lambda url: len(url), reverse=True)
                for url in urls:
                    text = text.replace(url, " URL ")
            if self.replace_numbers:
                text = re.sub(r'\d+(?:\.\d*)?(?:[eE][+-]?\d+)?', 'NUMBER', text)
            if self.remove_punctuation:
                text = re.sub(r'\W+', ' ', text, flags=re.M)
            word_counts = Counter(text.split())
            if self.stemming and stemmer is not None:
                stemmed_word_counts = Counter()
                for word, count in word_counts.items():
                    stemmed_word = stemmer.stem(word)
                    stemmed_word_counts[stemmed_word] += count
                word_counts = stemmed_word_counts
            X_transformed.append(word_counts)
        return np.array(X_transformed)

In [ ]:
#test transformer
X_few = X_train[:3]
X_few_wordcounts = EmailToWordCounterTransformer().fit_transform(X_few)
X_few_wordcounts

In [ ]:
#convert word counts to vectors by building  another transformer
#whose fit() method will build the vocabulary (an ordered list of the most common words) and whose transform() method will use the vocabulary to convert word counts to vectors. The output is a sparse matrix.
from scipy.sparse import csr_matrix

class WordCounterToVectorTransformer(BaseEstimator, TransformerMixin):
    def __init__(self, vocabulary_size=1000):
        self.vocabulary_size = vocabulary_size
    def fit(self, X, y=None):
        total_count = Counter()
        for word_count in X:
            for word, count in word_count.items():
                total_count[word] += min(count, 10)
        most_common = total_count.most_common()[:self.vocabulary_size]
        self.vocabulary_ = {word: index + 1 for index, (word, count) in enumerate(most_common)}
        return self
    def transform(self, X, y=None):
        rows = []
        cols = []
        data = []
        for row, word_count in enumerate(X):
            for word, count in word_count.items():
                rows.append(row)
                cols.append(self.vocabulary_.get(word, 0))
                data.append(count)
        return csr_matrix((data, (rows, cols)), shape=(len(X), self.vocabulary_size + 1))

In [ ]:
vocab_transformer = WordCounterToVectorTransformer(vocabulary_size=10)
X_few_vectors = vocab_transformer.fit_transform(X_few_wordcounts)
X_few_vectors

In [ ]:
X_few_vectors.toarray()

In [ ]:
vocab_transformer.vocabulary_

In [ ]:
#train our first spam classifier! Let's transform the whole dataset:
from sklearn.pipeline import Pipeline

preprocess_pipeline = Pipeline([
    ("email_to_wordcount", EmailToWordCounterTransformer()),
    ("wordcount_to_vector", WordCounterToVectorTransformer()),
])

X_train_transformed = preprocess_pipeline.fit_transform(X_train)

In [ ]:
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import cross_val_score

log_clf = LogisticRegression(solver="lbfgs", max_iter=1000, random_state=42)
score = cross_val_score(log_clf, X_train_transformed, y_train, cv=3, verbose=3)
score.mean()

In [ ]:
#print out the precision/recall we get on the test set:
from sklearn.metrics import precision_score, recall_score

X_test_transformed = preprocess_pipeline.transform(X_test)

log_clf = LogisticRegression(solver="lbfgs", max_iter=1000, random_state=42)
log_clf.fit(X_train_transformed, y_train)

y_pred = log_clf.predict(X_test_transformed)

print("Precision: {:.2f}%".format(100 * precision_score(y_test, y_pred)))
print("Recall: {:.2f}%".format(100 * recall_score(y_test, y_pred)))